---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [2]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df = pd.DataFrame(df,columns=["text"])
df

,text
0,03/25/93 Total time of visit (in minutes):\n
1,6/18/85 Primary Care Doctor:\n
2,sshe plans to move as of 7/8/71 In-Home Servic...
3,7 on 9/27/75 Audit C Score Current:\n
4,2/6/96 sleep studyPain Treatment Pain Level (N...
5,.Per 7/06/79 Movement D/O note:\n
6,"4, 5/18/78 Patient's thoughts about current su..."
7,10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8,3/7/86 SOS-10 Total Score:\n
9,(4/10/71)Score-1Audit C Score Current:\n


In [137]:
def date_sorter():
    import re
    import numpy as np
    from datetime import datetime
    doc = []
    with open('dates.txt') as file:
        for line in file:
            doc.append(line)

    df = pd.Series(doc)
    df = pd.DataFrame(df,columns=["text"])
    dic = {"jan":"1","feb":"2","mar":"2","apr":"4","may":"5","jun":"6","jul":"7","aug":"8","sep":"9",
           "oct":"10","nov":"11","dec":"12"}
    #i = 1
    # Your code here
    def getdates1(row):
        #print(row)
        #i+=1
        row = row.lower()
        ret = re.search(r"(\d{1,2}?[/-]?\d{1,2}[/-]\d{4})",row)
        if ret==None:
            ret = re.search(r"(\d{1,2}[/-]\d{1,2}[/-]\d{2})",row)
            if(ret!=None):
                ret = ret.group(0)
                yr = ret[-2:]
                yr = "19" + str(yr)
                ret = ret[:-2] +yr
                ret = ret.replace('-','/')
                
                return ret
            else:
                ret = re.search(r"((\d{1,2} )?(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z]* \d{4})",row)
                if(ret!=None):
                    ret = ret.group(0)
                    ret = ret.lower()
                    mword = re.search(r"[a-z]+",ret).group(0)
                    ret = ret.replace(mword,mword[0:3])
                    ret = ret.strip()
                    #print("this",ret)
                    if(ret[0].isalpha()):
                        ret = "01 "+ret
                    date_object = datetime.strptime(ret, '%d %b %Y').strftime('%m/%d/%Y')
                    ret = str(date_object)
                    
                    return ret
                else:
                    ret = re.search(r"((jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z]*[.]?( \d{1,2})?[,]? \d{4})",row)
                    if(ret!=None):
                        ret =ret.group(0)
                        ret = ret.replace('.','')
                        mword = re.search(r"[a-z]+",ret).group(0)
                        ret = ret.replace(mword,mword[0:3])
                        if(ret[3] == ','):
                            ret = ret[0:3] + " 01" +ret[3:]
                        if(ret.count(',') == 0):
                            ret = ret[0:6] +","+ret[6:]
                        date_object = datetime.strptime(ret, '%b %d, %Y').strftime('%m/%d/%Y')
                        ret = str(date_object)
                        #print(ret)
                        return ret
                    else:
                        ret = re.search(r"(\d?[/]?(19|20)(\d\d))",row).group(0)
                        if(len(ret)>4):
                            ret = ret[0:2]+"01/"+ret[2:]
                        else:
                            ret = "01/01/" + ret
                        #print(ret)
                        
                        return ret
                return None
        else:
            ret = ret.group(0)
            ret = ret.replace('-','/')
            if(ret.count('/') == 1):# & (str(ret[2])=="/"):
                if(str(ret[2])=="/"):
                    ret = ret[0:3]+"01/"+ret[3:]
                else:
                    #row = row.replace(ret,"")
                    ret = "08/01/2008"  
            return ret
    list = df["text"].apply(getdates1)#str.extract(r"(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})?((19|20)\d{2})?")
    #l = re.findall(r"(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})",df.loc[2]["text"])
    #print(list)
    #list[0] = list[0].fillna("") + list[1].fillna("")
    #list = list.drop([1,2],axis = 1)
    #list1 = df["text"].str.extract(r"") 7/8/71
    #
    #print(df.loc[2:6].values)
    def ctr(v):
        return  datetime.strptime(v, '%m/%d/%Y')
    list = list.apply(ctr)
    return np.argsort(list)
date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    439
486    255
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    427
495    231
496    141
497    186
498    161
499    413
Name: text, Length: 500, dtype: int64